In [92]:
import requests                
from bs4 import BeautifulSoup 
import pandas as pd
import re
import csv
import json
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


In [93]:
def extract_bv_id(url):
    match = re.search(r'video/(BV\w+)', url)
    if match:
        return match.group(1)  
    return None


In [94]:
def get_bulletcomment(link):
    '''
    given url, get the bullet comment of that video
    '''
    BV = extract_bv_id(link)
    #BV = re.search(r'/video/(BV\w+)/', link).group(1) # for each video it has an unique BV id in its url
    apilink = 'https://api.bilibili.com/x/player/pagelist?bvid=' + BV + '&jsonp=jsonp'

    header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
    response = requests.get(apilink, headers = header)
    cid = (response.json())['data'][0]['cid']

    cidlink = 'https://comment.bilibili.com/'+ str(cid) + '.xml'
    response = requests.get(cidlink, headers = header)
    soup = BeautifulSoup(response.text, "lxml-xml")
    texts = [d.text for d in soup.find_all('d')]
    return texts



#data = get_bulletcomment('https://www.bilibili.com/video/BV1wD421W7GP/?spm_id_from=333.1007.tianma.1-3-3.click&vd_source=a676e9574ae10f45ae8a73f5c6c428fd')


In [180]:
def get_bulletcomment_score(link):
    '''
    given url, get the bullet comment of that video
    '''
    BV = extract_bv_id(link)
    #BV = re.search(r'/video/(BV\w+)/', link).group(1) # for each video it has an unique BV id in its url
    apilink = 'https://api.bilibili.com/x/player/pagelist?bvid=' + BV + '&jsonp=jsonp'

    header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
    response = requests.get(apilink, headers = header)
    cid = (response.json())['data'][0]['cid']

    cidlink = 'https://comment.bilibili.com/'+ str(cid) + '.xml'
    response = requests.get(cidlink, headers = header)
    soup = BeautifulSoup(response.text, "lxml-xml")
    #texts = [d.text.encode('latin1').decode('utf-8') for d in soup.find_all('d')]

    d_tags = soup.find_all('d')
    output = []
    for tag in d_tags:
    # Split the 'p' attribute by commas and get the last element
        score = tag['p'].split(',')[-1]
        text = tag.text.encode('latin1').decode('utf-8')
        output.append([text, int(score)])

    return output

Go through a list of video to find porn seller

In [51]:
def get_related_video(link):
    '''
    given url, find a list of related short video
    '''
    header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
    response = requests.get(link, headers = header)

    soup = BeautifulSoup(response.text, 'html.parser')
    videos = soup.find_all('div', class_="video-page-card-small")
    video_links = []
    for video in videos:
        a_tag = video.find('a')
        if a_tag and 'href' in a_tag.attrs:
            video_links.append('https://www.bilibili.com' + a_tag['href'])
    return video_links



In [81]:
import time
import random

def get_n_related_video(link, n):
    '''
    given url, find a list of short video with size n 
    '''
    video_links = [link]
    BV = []
    current_link = link
    while len(video_links) < n:
        print(len(video_links))
        print(current_link)
        related_link = get_related_video(current_link)
        #for i in related_link:
        #    if extract_bv_id(i) not in BV:
        #        BV.append(extract_bv_id(i))
        #        video_links.append(i)
        #current_link = random.choice(related_link)

        if extract_bv_id(related_link[0]) not in BV:
            BV.append(extract_bv_id(related_link[0]))
            video_links.append(related_link[0])
            current_link = related_link[0]
        elif extract_bv_id(related_link[1]) not in BV:
            BV.append(extract_bv_id(related_link[1]))
            video_links.append(related_link[1])
            current_link = related_link[1]
        else:
            break
        
    

    return video_links

In [ ]:
videos = get_n_related_video('https://www.bilibili.com/video/BV1yFUJYsE5h/?spm_id_from=333.880.my_history.page.click&vd_source=a676e9574ae10f45ae8a73f5c6c428fd',300)
print(videos)

In [ ]:
def check_porn(links):
    '''
    check whether the videos inside a list contains the RIP as a bullet chat
    '''
    potential_porn = []
    j=0
    for link in links:
        print([j, len(potential_porn)])
        j+=1
        time.sleep(1)
        for i in get_bulletcomment(link):
            if i.encode('latin1').decode('utf-8') == '逝者安息':
                potential_porn.append(link)
    return(potential_porn)
        
potential_porn = check_porn(videos)

key word searching

In [122]:
def fetch_keyword_links(url):
    '''
    given a url (from key word searching), provide the links for all the video under this key word
    '''
    header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
    response = requests.get(url, headers = header)

    soup = BeautifulSoup(response.text, 'html.parser')
    videos = soup.find_all('div', class_="bili-video-card__info--right")
    video_links = []
    for video in videos:
        a_tag = video.find('a')
        if a_tag and 'href' in a_tag.attrs:
            video_links.append('https:' + a_tag['href'])
    return video_links
    

In [125]:
HK_riot = fetch_keyword_links("https://search.bilibili.com/all?keyword=%E9%A6%99%E6%B8%AF%E6%9A%B4%E4%B9%B1&from_source=webtop_search&spm_id_from=333.32")


In [126]:
len(HK_riot)

42

In [127]:
HK_riot[0]

'https://www.bilibili.com/video/BV1cJ411274r/'

In [170]:
def check_auth(url):
    '''
    given a video, check whether its uploader is an certified user
    '''
    try: 
        header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
        response = requests.get(url, headers = header)
        soup = BeautifulSoup(response.text, 'html.parser')
        user_info = soup.find('div', class_="up-detail-top")
        user_name = user_info.find('a').text
        user_type = user_info.find('a')['class']
        user_color = user_info.find('a')['style']
        if user_color == "color:#FB7299;":
            return "government"
        return "non_government"
    except:
        return "wrong"


    

In [171]:
check_auth(HK_riot[0])

'government'

check the HK_riot list, get all the comment, score, and user_type

In [184]:
HK_comment = []
for i in HK_riot:
    User = check_auth(i)
    if User != "wrong":
        HK_comment.append([User, get_bulletcomment_score(i)])
        

In [186]:
HK_comment

[['government',
  [['鲁迅先生说的好：我不惮以最坏的恶意，来揣测中国人……', 10],
   ['极  度  舒  适', 10],
   ['这枪声真好听，我倒回去听了十遍', 10],
   ['纣王:有种来忍受我的酷刑！', 10],
   ['可以出动战狼了', 10],
   ['枣庄脏话：奶奶b', 10],
   ['这个轨迹绝对绝对不是手动投掷!', 10],
   ['河南信求准备好嘞', 10],
   ['上面的，这真像三体中所描写的大低谷景象', 10],
   ['梁山帮请战', 10],
   ['宽容的人是否该对不宽容的人采取不宽容的态度？', 10],
   ['黑暗迪迦奥特曼请战！', 10],
   ['莱茵十字军请求出战！', 10],
   ['↓自我介绍', 10],
   ['国铁:再干老子直接把广深港高铁和广九直通车全停了', 10],
   ['这位大叔是清洁工，家境清寒而且重度烧伤！', 10],
   ['一条深圳河分出了天堂和地狱', 10],
   ['不到万不得已是不能开枪的，而且视频中的子弹用的是橡胶子弹，并非实弹，所以说爆头是不可能的', 10],
   ['张国荣在天之灵不会放过他们的', 10],
   ['它们必须付出代价', 10],
   ['这是敌方摄影师，视频是我们缴获来的', 10],
   ['把v的面具给老子摘下来！！！', 10],
   ['豫章书院：我觉得你们需要我（bishi）', 10],
   ['美国：拉我下水???', 10],
   ['我天', 10],
   ['他们都是快递驿站里的快递，有小件货大件货还有的没付清', 10],
   ['干的漂亮！', 10],
   ['愿逝者安息。。。。。。', 10],
   ['愿逝者安息', 10],
   ['不是人啊我靠', 10],
   ['广东脏话', 10],
   ['这他tm是人吗？', 10],
   ['好！！！', 10],
   ['山东脏话', 10],
   ['天津脏话', 10],
   ['好心疼啊！', 10],
   ['好样的', 10],
   ['希望把他们送死刑去。', 10],
   ['我tm的真的受不了了。', 10],
   ['江西脏话!!!

In [249]:
n=0
for i in HK_comment:
    n += len(i[1])
n


36534

In [217]:
from collections import Counter

b_values = [
    sub_item[0]  # 提取 B
    for item in HK_comment if isinstance(item, list) and len(item) > 1
    for sub_item in item[1] if isinstance(sub_item, list) and len(sub_item) > 0
]

# 统计每个 B 的出现次数
b_counts = Counter(b_values)

# 找出前十个最常见的 B
top_200_b_values = b_counts.most_common(200)

# 打印结果
print("Top 10 most common B values:")
for b, count in top_200_b_values:
    print(f"{b}: {count}")

Top 10 most common B values:
致敬: 430
666: 126
？？？: 84
下次一定: 80
？: 77
哈哈哈: 77
河南脏话: 72
逝者安息: 68
辛苦了: 67
广东脏话: 66
四川脏话: 64
？？？？？: 63
哈哈哈哈: 62
山东脏话: 61
广西脏话: 60
好: 59
哈哈哈哈哈: 58
福建脏话: 55
致敬！: 52
安徽脏话: 51
湖南脏话: 49
卧槽: 46
帅: 45
？？？？: 44
霸气: 44
漂亮: 40
哈哈哈哈哈哈哈: 40
哈哈哈哈哈哈: 40
江西脏话: 39
河北脏话: 38
泪目: 38
妙啊: 35
加油: 34
东北脏话: 32
湖北脏话: 32
天佑中华: 32
好！: 31
中华人民共和国万岁: 30
？？？？？？: 29
中国万岁: 29
哈哈: 29
说得好: 29
？？？？？？？: 28
江苏脏话: 28
贵州脏话: 27
☆☆☆★★: 27
恶心: 27
R.I.P: 26
2333: 26
浙江脏话: 25
哈哈哈哈哈哈哈哈: 25
一路走好: 24
重庆脏话: 24
笑死: 23
心疼: 22
人渣: 22
垃圾: 22
牛逼: 22
太帅了: 22
云南脏话: 21
哈哈哈哈哈哈哈哈哈: 21
游戏结束: 21
好帅: 20
干得漂亮: 19
?: 19
丧心病狂: 19
非人哉: 19
草: 19
龙门粗口: 19
谢谢: 19
？？: 18
光荣榜: 18
6666: 18
有: 18
中华人民共和国万岁！: 18
祖国万岁: 17
畜牲: 17
死刑: 17
不是人: 17
上海脏话: 17
狗: 17
该结束了: 17
枪毙: 16
陕西脏话: 16
北京脏话: 16
厚颜无耻: 16
1: 16
寒战3: 15
。: 15
汉奸: 15
对: 15
死亡名单: 15
大快人心: 14
我爱中国: 14
？？？？？？？？？？: 14
乌合之众: 14
过街老鼠: 14
辽宁脏话: 13
中国加油: 13
？？？？？？？？？？？: 13
口吐芬芳: 13
？？？？？？？？: 13
好丑: 13
政审: 13
支持: 13
太轻了: 13
杨安泽: 13
丑人多作怪: 13
甘肃脏话: 13
帅啊: 13
好好好: 12
疯了: 12
宁夏脏话: 1

TEST: 脏话，卧槽， 恶心， 人渣， 垃圾， 草， 丧心病狂， 狗， 枪毙， 汉奸， 杀， 神经病， 死

In [248]:
# 指定的 B*（允许部分匹配）
target_b = "心疼"

# 初始化存储最高值的字典
results = {"government": [], "non_government": []}

# 计算每个元素中包含 target_b 的最高 C 值并分类
for item in HK_comment:
    # 检查是否符合 [A, [[B, C], ...]] 的结构
    if isinstance(item, list) and len(item) == 2 and isinstance(item[1], list):
        # 提取 A 的值
        government_status = item[0]
        
        # 查找包含 target_b 的最高 C 值
        max_c = max(
            [sub_item[1] for sub_item in item[1] if target_b in sub_item[0]], 
            default=None  # 如果没有匹配，返回 None
        )
        
        # 如果找到了 C 值，添加到对应类型中
        if max_c is not None:
            results[government_status].append(max_c)

# 计算平均值
average_government = sum(results["government"]) / len(results["government"]) if results["government"] else 0
average_non_government = sum(results["non_government"]) / len(results["non_government"]) if results["non_government"] else 0

#average_government = sum(results["government"]) / 17 if results["government"] else 0
#average_non_government = sum(results["non_government"]) / 24 if results["non_government"] else 0

# 打印结果
print(f"Average highest value for '{target_b}' in government: {average_government:.2f}")
print(f"Average highest value for '{target_b}' in non_government: {average_non_government:.2f}")

Average highest value for '心疼' in government: 9.55
Average highest value for '心疼' in non_government: 8.50


In [208]:
G = 0
N = 0
for i in HK_comment:
    if i[0] == "government":
        G+=1
    if i[0] == "non_government":
        N+=1
print (G,N)

17 24


Weekly Top word

In [57]:
def fetch_weekly_video_links(url):
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    
    try:
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        video_links = []
        
        videos = soup.find_all('div', class_="video-card__content")
        for video in videos:
            a_tag = video.find('a')
            if a_tag and 'href' in a_tag.attrs:
                video_links.append(a_tag['href'])

    finally:
        driver.quit()
    
    return video_links



In [ ]:
url = "https://www.bilibili.com/v/popular/weekly?num=267"
links = fetch_video_links(url)

In [44]:
data_processed = []
for i in links:
    data = get_bulletcomment(i)
    for j in data:
        j = j.encode('latin1').decode('utf-8')
        data_processed.append(j)
df = pd.DataFrame(data_processed)
df.to_csv('超大弹幕', index=False, header=None, encoding="utf_8_sig")
print("写入文件成功")


写入文件成功


In [ ]:
from collections import Counter

def find_top_duplicates(filename, top_n=100):
    with open(filename, 'r', encoding='utf-8') as file:
        # 读取所有行，并去除两端的空白字符（如换行符）
        lines = [line.strip() for line in file if line.strip()]
    
    # 使用Counter统计每行出现的次数
    line_counts = Counter(lines)
    
    # 获取出现次数最多的top_n行
    top_lines = line_counts.most_common(top_n)
    
    return top_lines

# 调用函数，假设文件名为'超大弹幕.txt'
top_duplicates = find_top_duplicates('超大弹幕')
for line, count in top_duplicates:
    print(f'"{line}" 出现了 {count} 次')

In [ ]:
##test
data_processed = []
for i in data:
    i = i.encode('latin1').decode('utf-8')
    data_processed.append(i)
df = pd.DataFrame(data_processed)
df.to_csv('弹幕', index=False, header=None, encoding="utf_8_sig")
print("写入文件成功")


In [52]:
gz = get_bulletcomment('https://www.bilibili.com/video/BV1Ym411S7sq/?spm_id_from=333.337.search-card.all.click&vd_source=a676e9574ae10f45ae8a73f5c6c428fd')
#data_processed = []
for i in gz:
    i = i.encode('latin1').decode('utf-8')
    data_processed.append(i)
df = pd.DataFrame(data_processed)
df.to_csv('贵州', index=False, header=None, encoding="utf_8_sig")
print("写入文件成功")
top_duplicates = find_top_duplicates('贵州')
for line, count in top_duplicates:
    print(f'"{line}" shows {count} times')

写入文件成功
"嗯" shows 214 times
"万一刚好差我一个怎么办" shows 73 times
"顶" shows 72 times
"万一就差我一个呢" shows 56 times
"今日我若冷眼旁观，他日亦无人为我摇旗呐喊！" shows 52 times
"加油" shows 50 times
"我本来可以选择沉默，但是万一刚好差我一个呢" shows 46 times
"今日我若冷眼旁观，他日亦无人为我摇旗呐喊" shows 38 times
"支持" shows 36 times
"星星之火 可以燎原！坚持发声！" shows 34 times
"我不怕我是多余的那个，我就怕我正好缺少的那一个" shows 33 times
"节哀" shows 32 times
"今日我若冷眼旁观，他日亦无人为我摇旗呐喊！！！" shows 28 times
"星星之火可以燎原" shows 26 times
"人民不是儿戏" shows 25 times
"星星之火，可以燎原！" shows 25 times
"星星之火，可以燎原" shows 25 times
"今日若我冷眼旁观，他日亦无人为我摇旗呐喊！" shows 20 times
"星星之火，可以燎原。" shows 19 times
"节哀顺变" shows 17 times
"我可以选择沉默，但我不希望刚好就差我一个" shows 14 times
"星星之火，可以燎原，坚持发声！" shows 13 times
"今日若我冷眼旁观，他日祸临己身，则无人为我摇旗呐喊" shows 13 times
"星星之火可以燎原！" shows 12 times
"万一刚好差我一个呢" shows 11 times
"星星之火，可以燎原！坚持发声！" shows 10 times
"我可以选择沉默，但我不希望刚好就差我" shows 10 times
"今日我等若冷眼旁观，他日祸临己身，则无人为我摇旗呐喊" shows 10 times
"顶顶顶" shows 10 times
"星星之火可以燎原！支持发声！坚持发声！" shows 9 times
"万一差我一个呢" shows 9 times
"星星之火 可以燎原" shows 9 times
"我本来可以默不成声的，但如果只差我一个呢？" sh

In [176]:
def get_censorscore(link):
    '''
    given url, get the bullet comment of that video
    '''
    BV = extract_bv_id(link)
    #BV = re.search(r'/video/(BV\w+)/', link).group(1) # for each video it has an unique BV id in its url
    apilink = 'https://api.bilibili.com/x/player/pagelist?bvid=' + BV + '&jsonp=jsonp'

    header = { "User-Agent" : "demo scraper for teaching purposes ys@u.edu" } 
    response = requests.get(apilink, headers = header)
    cid = (response.json())['data'][0]['cid']

    cidlink = 'https://comment.bilibili.com/'+ str(cid) + '.xml'
    response = requests.get(cidlink, headers = header)
    soup = BeautifulSoup(response.text, "lxml-xml")
    d_tags = soup.find_all('d')
    scores = []
    for tag in d_tags:
    # Split the 'p' attribute by commas and get the last element
        score = tag['p'].split(',')[-1]
        scores.append(int(score))

    return scores

In [87]:
s = get_censorscore('https://www.bilibili.com/video/BV1rCsdeEEBT/?spm_id_from=333.337.search-card.all.click&vd_source=a676e9574ae10f45ae8a73f5c6c428fd')


In [ ]:
import matplotlib.pyplot as plt

counts = {x: s.count(x) for x in set(s)}
total = len(s)
percentages = {k: v / total * 100 for k, v in counts.items()}

# X and Y data for the plot
x = list(percentages.keys())
y = list(percentages.values())

# Create bar plot
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='skyblue')
plt.title('Guanca.cn Censor Score Distribution')
plt.xlabel('Censor Score')
plt.ylabel('Percentage (%)')
plt.xticks(range(0, max(x) + 1))  # Set x-ticks to start from 0 and go beyond the maximum x value
plt.xlim(0, max(x) + 1)  # Set the x-axis limits to start at 0 and extend beyond the max value
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add horizontal grid lines for better readability
plt.show()

In [91]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Automatically download and use the correct ChromeDriver
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver with the Service
driver = webdriver.Chrome(service=service)

# Your browser automation code here
driver.get("https://www.bilibili.com/blackroom/ban")

# 模拟滚动加载
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 滚动到底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)  # 等待页面加载
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # 判断是否还有新内容加载
        break
    last_height = new_height

# 抓取页面数据
content = driver.page_source
print(content)

driver.quit()


OSError: [Errno 8] Exec format error: '/Users/sys/.wdm/drivers/chromedriver/mac64/131.0.6778.87/chromedriver-mac-x64/THIRD_PARTY_NOTICES.chromedriver'